### Chat with Video Demo
The notebook demonstrates how to run and use example chat with video application, a multimodal chat application that allows users to upload a video and ask questions about the video content based on the visual and audio information.

The first step is to start the application. This is done by running the following command:

```bash
HF_TOKEN="<your token>" CUDA_VISIBLE_DEVICES="0" aana deploy aana.projects.chat_with_video.app:aana_app
```

[API documentation](https://127.0.0.1:8000/redoc) describes what endpoints are available.

A video can be sent to the indexing endpoint `/video/index_stream`.
This will extract the audio transcription and captions for the frames and store them in the database for later retrieval. The following code snippet shows how to send the video to the indexing endpoint.

The indexing endpoint is a streaming endpoint which means that it responds with a stream of data. Set `stream=True` in the request to get the response as a stream. The endpoint return captions and audio transcription as soon as they are available which allows you to create more responsive applications by showing the captions as they are generated.

In [2]:
import json

import requests

video = {
    "url": "https://www.youtube.com/watch?v=wxN1T1uxQ2g",  # Video URL, Aana SDK supports URLs (including YouTube), file paths or even raw video data
    "media_id": "wxN1T1uxQ2g",  # Media ID, so we can ask questions about the video later by using this ID
}

data = {
    "video_params": {
        "fast_mode_enabled": True,  # Enable fast mode, which only processes keyframes
    },
    "video": video,
}

url = "http://127.0.0.1:8000/video/index_stream"
response = requests.post(url, data={"body": json.dumps(data)}, stream=True)

for chunk in response.iter_content(chunk_size=None):
    json_data = json.loads(chunk)
    if "error" in json_data:
        print(json_data["error"])
        print(json_data["message"])
        print(json_data["stacktrace"])
        break
    else:
        print(json_data)

{'media_id': 'wxN1T1uxQ2g', 'metadata': {'title': 'Everything Everywhere All At Once | Official Trailer HD | A24', 'description': "SUBSCRIBE: http://bit.ly/A24subscribe\r\n\r\nA film from Daniels and starring Michelle Yeoh, Ke Huy Quan, Stephanie Hsu, Jenny Slate, Harry Shum Jr., with James Hong and Jamie Lee Curtis. EVERYTHING EVERYWHERE ALL AT ONCE – In Theaters March 25, 2022! #EverythingEverywhere\r\n\r\nHow to Watch: https://bit.ly/EEAAO-OnDemand\n\nRELEASE DATE: March 25, 2022\r\nDIRECTOR: Daniels\r\nCAST: Michelle Yeoh, Stephanie Hsu, Ke Huy Quan, Jenny Slate, Harry Shum Jr., with James Hong and Jamie Lee Curtis\r\n\r\nFollow Everything Everywhere on Instagram: https://bit.ly/EverythingEverywhere_IG\r\nFollow Everything Everywhere on Twitter: https://bit.ly/EverythingEverywhere_TW\r\nLike Everything Everywhere on Facebook: https://bit.ly/EverythingEverywhere_FB\r\n\r\n------\r\n\r\nABOUT A24:\r\nThe studio behind MOONLIGHT, LADY BIRD, EX MACHINA, THE WITCH, EIGHTH GRADE, HEREDIT

Once the video is indexed, you can start asking questions about the video content. The following code snippet shows how to send a question to the `/video/chat_stream` endpoint. The endpoint is also a streaming endpoint and returns tokens of the answer as it's being generated. This allows you to show the answer as it's being generated for a more responsive user experience.

In [2]:
media_id = video["media_id"]

question = (
    "What is happening in this video? "
    "Focus on highlighting the story, the narrative, emotionally and thematically significant aspects. "
    "Do not use bulleted list in your answer."
)

data = {
    "question": question,
    "media_id": media_id,
    "sampling_params": {
        "temperature": 0.9,
        "max_tokens": 1024,
    },
}

url = "http://127.0.0.1:8000/video/chat_stream"

response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
for chunk in response.iter_content(chunk_size=None):
    chunk_dict = json.loads(chunk)
    if "error" in chunk_dict:
        print(chunk_dict["error"])
        print(chunk_dict["message"])
        print(chunk_dict["stacktrace"])
        break
    token = chunk_dict["completion"]
    print(token, end="")

Based on the analysis, this video appears to be a trailer for a film, specifically the official trailer for "Everything Everywhere All At Once" directed by Daniel Kwan and Daniel Scheinert. The visuals depict a multitude of scenes showcasing various characters and settings, which are likely meant to represent different parallel universes or alternate realities. The audio transcript contains cryptic and surreal statements that seem to tie into the film's theme of exploring the multiverse and alternate realities. The trailer appears to be building up to a mysterious and intriguing narrative that explores the idea of multiple versions of a person and the consequences of their choices.